# Multi-Agent AI System - Interactive Demo

This notebook demonstrates the core capabilities of the Multi-Agent AI System:
1. Document ingestion and RAG queries
2. Multi-agent orchestration
3. Specialized agent tasks (research, analysis, code)

## Setup

In [ ]:
# Install dependencies if needed
# !pip install -r requirements.txt

In [ ]:
import os
import sys
from pathlib import Path

# Add project root to path
project_root = Path.cwd().parent
sys.path.insert(0, str(project_root))

# Load environment variables
from dotenv import load_dotenv
load_dotenv()

print("✅ Environment loaded")

## 1. Initialize the RAG Pipeline

First, let's set up the RAG pipeline for document ingestion and retrieval.

In [ ]:
from src.rag import RAGPipeline, Document, ChunkingStrategy
from src.rag.embeddings import get_embedding_model

# Initialize embedding model
embedding_model = get_embedding_model(provider="openai")
print(f"📊 Embedding model: {embedding_model.model_name}")
print(f"   Dimension: {embedding_model.dimension}")

# Initialize RAG pipeline
rag = RAGPipeline(
    embedding_model=embedding_model,
    collection_name="demo_collection"
)

print("\n✅ RAG Pipeline initialized")

### Ingest Sample Documents

In [ ]:
# Sample documents for demonstration
sample_documents = [
    Document(
        content="""
        Multi-Agent AI Systems represent a paradigm shift in how we build intelligent applications.
        Unlike monolithic AI systems, multi-agent architectures decompose complex tasks into 
        specialized subtasks handled by different agents. Each agent has specific capabilities
        and can communicate with other agents to solve problems collaboratively.
        
        Key benefits include:
        - Modularity: Each agent can be developed and tested independently
        - Scalability: New agents can be added without affecting existing ones
        - Specialization: Agents can be optimized for specific task types
        - Robustness: System continues functioning if one agent fails
        """,
        metadata={"source": "architecture_guide", "topic": "multi-agent systems"}
    ),
    Document(
        content="""
        RAG (Retrieval-Augmented Generation) combines the power of retrieval systems with
        large language models. Instead of relying solely on parametric knowledge, RAG systems
        retrieve relevant documents from a knowledge base and use them to ground responses.
        
        The typical RAG pipeline includes:
        1. Document chunking: Breaking documents into manageable pieces
        2. Embedding: Converting text to vector representations
        3. Indexing: Storing vectors in a searchable database
        4. Retrieval: Finding relevant chunks for a query
        5. Generation: Using retrieved context to generate responses
        """,
        metadata={"source": "rag_documentation", "topic": "RAG"}
    ),
    Document(
        content="""
        Best practices for prompt engineering with multi-agent systems:
        
        1. Clear Role Definition: Each agent should have a well-defined role and capabilities
        2. Structured Outputs: Use JSON or structured formats for inter-agent communication
        3. Error Handling: Agents should gracefully handle failures and provide feedback
        4. Context Management: Carefully manage context to stay within token limits
        5. Tracing: Implement comprehensive logging for debugging and optimization
        """,
        metadata={"source": "best_practices", "topic": "prompt engineering"}
    )
]

# Ingest documents
import asyncio

async def ingest():
    result = await rag.ingest_documents(
        documents=sample_documents,
        chunking_strategy=ChunkingStrategy.RECURSIVE
    )
    return result

ingestion_result = asyncio.get_event_loop().run_until_complete(ingest())
print(f"📚 Ingested documents:")
print(f"   Documents: {ingestion_result['documents_processed']}")
print(f"   Chunks: {ingestion_result['chunks_created']}")
print(f"   Embeddings: {ingestion_result['embeddings_generated']}")

### Query the Knowledge Base

In [ ]:
async def search(query: str):
    result = await rag.query(query, top_k=3)
    return result

# Example query
query = "What are the key benefits of multi-agent systems?"
query_result = asyncio.get_event_loop().run_until_complete(search(query))

print(f"🔍 Query: {query}\n")
print("📄 Retrieved Documents:")
for i, (doc, score) in enumerate(zip(query_result.documents, query_result.scores)):
    print(f"\n--- Result {i+1} (Score: {score:.3f}) ---")
    print(f"Source: {doc.metadata.get('source', 'unknown')}")
    print(f"Content: {doc.content[:200]}...")

## 2. Initialize Multi-Agent System

Now let's set up the multi-agent orchestrator with specialized agents.

In [ ]:
from src.agents import OrchestratorAgent, ResearchAgent, AnalystAgent, CodeAgent
from src.core import get_llm, ConversationContext

# Initialize LLM
llm = get_llm(provider="openai", model="gpt-4-turbo")

# Initialize specialized agents
research_agent = ResearchAgent(llm=llm, rag_pipeline=rag)
analyst_agent = AnalystAgent(llm=llm)
code_agent = CodeAgent(llm=llm, enable_execution=True)

# Initialize orchestrator
orchestrator = OrchestratorAgent(
    llm=llm,
    agents={
        "research": research_agent,
        "analyst": analyst_agent,
        "code": code_agent
    }
)

print("🤖 Multi-Agent System initialized!")
print(f"   Available agents: {list(orchestrator.agents.keys())}")

## 3. Execute Multi-Agent Workflows

### Example 1: Research Task

In [ ]:
# Research task
research_task = "Explain the benefits of RAG in multi-agent systems and how they work together"

async def run_research():
    context = ConversationContext()
    result = await orchestrator.execute_workflow(research_task, context)
    return result

research_result = asyncio.get_event_loop().run_until_complete(run_research())

print("🔬 Research Task Result:")
print(f"   Success: {research_result.success}")
print(f"   Execution Time: {research_result.total_time:.2f}s")
print(f"   Agents Used: {research_result.execution_order}")
print(f"\n📝 Response:\n{research_result.final_output}")

### Example 2: Code Generation Task

In [ ]:
# Direct code agent usage
code_task = "Write a Python function that implements exponential backoff retry logic"

async def run_code_task():
    context = ConversationContext()
    result = await code_agent.execute(code_task, context)
    return result

code_result = asyncio.get_event_loop().run_until_complete(run_code_task())

print("💻 Code Generation Result:")
print(f"   Success: {code_result.success}")
print(f"   Execution Time: {code_result.execution_time:.2f}s")
if code_result.success and code_result.output:
    print(f"\n📝 Generated Code:")
    print(code_result.output.get('code', 'No code generated'))

### Example 3: Analysis Task

In [ ]:
# Sample data for analysis
sample_data = {
    "monthly_users": [1000, 1200, 1500, 1400, 1800, 2200, 2500],
    "conversion_rate": [0.02, 0.025, 0.022, 0.028, 0.03, 0.032, 0.035],
    "months": ["Jan", "Feb", "Mar", "Apr", "May", "Jun", "Jul"]
}

analysis_task = "Analyze the user growth and conversion rate trends. What recommendations do you have?"

async def run_analysis():
    context = ConversationContext()
    result = await analyst_agent.execute(
        analysis_task, 
        context,
        data=sample_data
    )
    return result

analysis_result = asyncio.get_event_loop().run_until_complete(run_analysis())

print("📊 Analysis Result:")
print(f"   Success: {analysis_result.success}")
print(f"   Execution Time: {analysis_result.execution_time:.2f}s")
if analysis_result.success and analysis_result.output:
    output = analysis_result.output
    print(f"\n📈 Summary: {output.get('summary', 'N/A')}")
    if 'recommendations' in output:
        print(f"\n💡 Recommendations:")
        for rec in output['recommendations']:
            print(f"   - {rec.get('action', rec)}")

## 4. Complex Multi-Agent Workflow

Let's execute a complex task that requires multiple agents to collaborate.

In [ ]:
complex_task = """
I need help building a data pipeline:
1. First, research best practices for ETL pipelines
2. Then analyze the trade-offs between batch and stream processing
3. Finally, generate Python code for a simple ETL pipeline skeleton
"""

async def run_complex_workflow():
    context = ConversationContext()
    result = await orchestrator.execute_workflow(complex_task, context)
    return result

complex_result = asyncio.get_event_loop().run_until_complete(run_complex_workflow())

print("🚀 Complex Workflow Result:")
print(f"   Success: {complex_result.success}")
print(f"   Total Time: {complex_result.total_time:.2f}s")
print(f"   Execution Order: {complex_result.execution_order}")
print(f"\n📋 Agent Results:")
for agent_name, result in complex_result.agent_results.items():
    print(f"\n   {agent_name}:")
    print(f"      Success: {result.get('success', 'N/A')}")
    print(f"      Time: {result.get('execution_time', 0):.2f}s")

print(f"\n📝 Final Output:\n{complex_result.final_output[:1000]}...")

## 5. Cleanup

In [ ]:
# Optional: Delete the demo collection
# rag.delete_collection()
# print("🧹 Demo collection deleted")

print("\n✅ Demo complete!")
print("\nNext steps:")
print("  1. Try different queries with the RAG pipeline")
print("  2. Experiment with different agent combinations")
print("  3. Ingest your own documents into the knowledge base")
print("  4. Build custom tools for your agents")